# 数据清洗与整理

## 数据介绍

**station.csv** : 记录了美国旧金山海湾地区共享自行车站的信息，包含以下变量：
- **id** :  车站编号
- **name** :  车站名称
- **lat** :  车站的纬度
- **long** :  车站的经度
- **docks** :  车站的码头数
- **city** :  车站所在位置

** trips1.csv、trips2.csv、trips3.csv** : 2013年8月底的所有trips, 包含以下变量：
- **start_id** : trip的起始点
- **end_id** : trip的终止点
- **start_date** : trip的起始时间
- **end_date** : trip的终止时间
- **subscription_type** : 使用者类型（ 用户“Subscriber” 或者客户“Customer” ）


### 数据处理过程将会完成以下各项：
1. 导入 'station.csv' 数据，并且命名为 stations<br>
   导入 'trips1.csv'、'trips2.csv'、'trips3.csv' 数据，并且分别命名为 trips1、trips2、trips3
2. 将 trips1、trips2、trips3 合并为一个Dataframe, 命名为 trips
3. 将 staions 中所有列名称前添加字段'start_'，并且将 start_id 设置为列索引
4. 将 trips 和 stations 按照起始车站id进行字段匹配并合并，保留所有匹配成功的信息
5. 将trips_stations 导出为' trips_stations.csv'文件
6. 查看 trips_stations 中是否包含有重复值，并且将重复值删除
7. 查看 trips_stations 中是否包含有缺失值，并且处理缺失值
8. 去除 trips_stations 中 ‘start_name’ 列中每个字符串左右两边的空格和’#‘
9. 将 'start_date' 和 'end_date' 中日期和时间进行拆分, 并分别记录在 'start_date'、'start_time'、'end_date'、'end_time'列
10. 将每个终点车站数进行分组，分为<br>
    '13以下', '13到15', '15到17', '17到19','19到21','21到23','23到25','25以上' 几类，<br>
   并且在'start_docks'列右侧增加一列'start_docks_classification'记录每个车站数所属的分类 
11. 将'subscription_type'转化为虚拟变量，添加在dateframe的最后一列

### 导入数据

#### 1. 导入 'station.csv' 数据，并且命名为 stations <br> <br>导入 'trips1.csv'、'trips2.csv'、'trips3.csv' 数据，并且分别命名为 trips1、trips2、trips3

In [3]:
# 导入 pandas 和 numpy 函数库
import pandas as pd
import numpy as np

In [4]:
# 导入csv文件
stations = pd.read_csv('./data/stations.csv', encoding= 'utf-8' )
trips1 = pd.read_csv('./data/trips1.csv', encoding = 'utf-8')
trips2 = pd.read_csv('./data/trips2.csv', encoding = 'utf-8')
trips3 = pd.read_csv('./data/trips3.csv', encoding = 'utf-8')
type(trips1)



pandas.core.frame.DataFrame

read_csv()参数列表：<br>
sep 指定分隔符 默认为','<br>
names 指定列名列表,相当于给没有列标题的csv添加列索引

### 合并数据

#### 2. 将 trips1、trips2、trips3 合并为一个Dataframe, 命名为 trips

In [ ]:
trips = pd.concat([trips1,trips2,trips3])
trips

行之间的连接

join = 'inner' 取两表的交集<br>
join = 'outer 取两表的并集

concat()的轴参数：<br>
    axis = 0 列对齐，两表上下合并，每一列是增长了<br>
    axis = 1 行对齐，两表左右合并，每一行是增长了

#### 3. 将 staions 中所有列名称前添加字段'start_'，并且将 start_id 设置为列索引

In [ ]:
# 修改列名称
stations.columns #pandas index数组，表示列索引的集合
stations.columns = stations.columns.map(lambda x: 'start_' + x) #注意要把map之后的赋值过去
print(stations.columns)
print(trips.columns)

map() 会根据提供的函数对指定序列做映射。

In [ ]:
# 将 start_id 设置为列索引
stations.set_index(['start_id'], inplace = True) #这里set_index也没有原地修改，不过inplace可以指定它原地修改
stations

#### 4. 将 trips 和 stations 按照起始车站id进行字段匹配并合并，保留所有匹配成功的信息

In [ ]:
# 将 trips 和 stations 按照起始车站id进行匹配
#别忘了merge就是增加新列
trips_stations = pd.merge(trips, stations, left_on = 'start_id',right_on = 'start_id',
                          how = 'left',right_index=True) #因为右边数据集使用的是index进行匹配，所以要加一个right_index参数
#也要新加一个变量记录活动
trips_stations

how = 'inner'(默认）未完全匹配的时候，保留已匹配的部分<br>
how = 'left' 未完全匹配的时候，保留左边未匹配的部分<br>
how = 'right'未完全匹配的时候，保留右边未匹配的部分<br>
how = 'outer'未完全匹配的时候，保留两边所有未匹配的部分

In [ ]:
trips.join(stations,how = 'left')

### 导出数据

#### 5. 将trips_stations 导出为' trips_stations.csv'文件

In [ ]:
# 导出csv文件
trips_stations.to_csv('trips_stations.csv')

### 去除重复值

#### 6. 查看 trips_stations 中是否包含有重复值，并且将重复值删除

只有某一行的所有元素与之前的某行完全相同时，才判断为相同。

In [ ]:
# 查看'stations'中是否存在重复值
trips_stations_dup = trips_stations.duplicated() #返回一个series，索引是原来的行索引，值是True/False
trips_stations[trips_stations_dup] #条件检索
trips_stations_dup

In [ ]:
# 删除重复值
trips_stations = trips_stations.drop_duplicates()

In [ ]:
# 检查重复值是否已被删除
trips_stations_dup = trips_stations.duplicated()
trips_stations_dup

### 处理缺失值

#### 7. 查看 trips_stations 中是否包含有缺失值，并且处理缺失值

有时将缺失值得样本剔除，有时填充

In [ ]:
# 查看’trips'文件中是否含有缺失值
isNA_trips_stations = trips_stations.isnull() #返回dataframe，每个元素都是True/False
cond_na_row = isNA_trips_stations.any(axis=1)   #.any(axis=)方法，沿轴做或运算，只要该轴有一个True/Nonzero/...值就把这行判定为True,返回boolean series
trips_stations[cond_na_row]

In [ ]:
# 向上填充缺失值,用它同列上面的元素进行填充。非原地操作。返回series数组标识该列的值，再赋予该列。
trips_stations['start_docks'] = trips_stations['start_docks'].ffill()

In [ ]:
# 向下填充缺失值。非原地操作。记得把填充后的列series赋给原列。
trips_stations['start_docks'] = trips_stations['start_docks'].bfill()


In [ ]:
# 中位数填充缺失值
docks_median = trips_stations['start_docks'].median()
trips_stations['start_docks'].fillna(docks_median)

In [ ]:
# 修改原数据
docks_median = trips_stations['start_docks'].median
trips_stations['start_docks'] = trips_stations['start_docks'].fillna(docks_median)

In [ ]:
# 将 lat、long 中的缺失值设置为‘未知’
trips_stations['start_lat'] = trips_stations['start_lat'].fillna('未知')
trips_stations['start_long'] = trips_stations['start_long'].fillna('未知')

In [ ]:
# 删除缺失值的样本
trips_stations = trips_stations.dropna()

In [ ]:
# 检查’trips'中的缺失值是否已被删除
isNA_trips_stations = trips_stations.isnull()
trips_stations[isNA_trips_stations.any(axis=1)]

### 处理空格值

#### 8. 去除 trips_stations 中 ‘start_name’ 列中每个字符串左右两边的空格和’#‘

对series数组使用.str()方法，可以把每个元素转换成字符串，然后用.strip()/.lstrip()/.rstrip()对每一个元素删除特定值

In [ ]:
# 删除'name'列中每个字符串左边的空格(开头空格）
trips_stations['start_name'].str.strip()

In [ ]:
# 删除'name'列中每个字符串右边的空格(尾部空格)
trips_stations['start_name'].str.rstrip()

In [ ]:
# 删除'name'列中每个字符串左右两边的空格(头尾空格)，并且修改原数据
trips_stations['start_name'] = trips_stations['start_name'].str.strip()

In [ ]:
# 删除字符串左边的‘#’
trips_stations['start_name'].str.lstrip('#')

In [ ]:
# 删除字符串左边的'#'
trips_stations['start_name'].str.rstrip('#')

In [ ]:
# 删除字符串两边的'#',并且修改原数据
trips_stations['start_name'] = trips_stations['start_name'].str.strip('#')

### 字段拆分

就是把一个字段（一列）按照某种规则竖着劈成两个或多个部分

#### 9. 将 'start_date' 和 'end_date' 中日期和时间进行拆分, 并分别记录在 'start_date'、'start_time'、'end_date'、'end_time'列中

In [ ]:
# 将'start_date'按照空格拆分成两列
new_col_start = trips_stations['start_date'].str.split(' ', 1, True) #True表示返回dataframe, False表示返回series
# 设置列名
new_col_start.columns = ['start_date','start_time']
# 将'trips_station'中'start_date'更改为new_col_start中的第一列
trips_stations['start_date'] = new_col_start['start_date']
# 在'start_date'右侧新增一列，记录'start_time'
trips_stations.insert(loc = 3, column = 'start_time', value = new_col_start['start_time'])

trips_stations.head()

In [ ]:
# 将'end_date'按照空格拆分成两列
new_col_end = trips_stations['end_date'].str.split(' ', 1, True)
print(new_col_end)
# 设置列名
new_col_end.columns = ['end_date','end_time']
# 将'trips_station'中'end_date'更改为new_col_end中的第一列
trips_stations['end_date'] = new_col_end['end_date']
# 在'end_date'右侧添加一列，记录'end_time'
trips_stations.insert(
    loc = 5, column = 'end_time', 
    value = new_col_end['end_time'])

trips_stations.head()

### 数据分组

#### 10.  将每个终点车站数进行分组，分为 <br>  
####  '13以下', '13到15', '15到17', '17到19', '19到21', '21到23', '23到25', '25以上' 几类，<br>
#### 并且在'start_docks'列右侧增加一列'start_docks_classification'记录每个车站数所属的分类 

In [ ]:
# 输出'start_docks'的最小值和最大值
print(min(trips_stations.start_docks), max(trips_stations.start_docks))

In [ ]:
# 设置分组边界
bins = [
    min(trips_stations.start_docks)-1, 13, 15, 17, 19, 21, 23, 25,
    max(trips_stations.start_docks)+1
]

In [ ]:
# 按照分组边界对'start_docks'进行分组
cut = pd.cut(trips_stations.start_docks, bins, right=False)

In [ ]:
# 设置每个组的label
labels = ['13以下', '13到15', '15到17', '17到19','19到21','21到23','23到25','25以上']

In [ ]:
# 用'labels'代替数字
cut = pd.cut(trips_stations.start_docks, bins, right=False, labels = labels) #right表示右边界是否包含,labels表示label
cut

In [ ]:
# 将列名转化为列名列表
col_name = trips_stations.columns.tolist() #直接columns是Index数组类型
# 在'start_docks'右侧添加一列，记录分组结果
trips_stations.insert(
    loc = col_name.index('start_docks')+1,  #如何很快得到想添加的列索引位置，省的一列列数过来。列表的index方法返回元素下标。
    column = 'start_docks_classification',  #给插入列设置列索引
    value = cut                             #列值
)
trips_stations.head()

# 添加虚拟变量（用编码代表非数值的值）

#### 11. 将'subscription_type'转化为虚拟变量，添加在dateframe的最后一列

In [ ]:
# 转化’subscription_type'为虚拟变量
trips_stations_dummies = pd.get_dummies(
    trips_stations,
    columns = ['subscription_type'],
    prefix=['subscription_type'],
    prefix_sep="_",  #sep后面是非数值的值，也就是重新定义列索引为：原列索引_值
    dummy_na=False,
    drop_first=False
)

trips_stations_dummies['subscription_type'] = trips_stations['subscription_type']
# prefix 在没有列标题时，给列添加前缀

In [ ]:
trips_stations_dummies.head()